In [ ]:
# Filtered Calpads 8.1 data to Start Dates after 2021-07-30 effectively getting 2021-2022 school year schools. 
# Frame has one record per student. Then pass all singular SSIDs to API call. 
# 10820 rows

# SSIDs are passed in to CALPADS API for student enrollment, and then filtered for time period between 2020-08-01 : 2021-07-30 
# This effectively gets prior enrollments before 2021-2022 school year. 
# 10548 rows

# Filtered to most recent school from CALPADS API call simplifying to one school per SSID. Then merged on common SSID between 8.1 datq
# and Calpads API call. 

# Note: API call had 272 rows less. (10820 rows - 10548 rows)
# Final frame has 10548 due to merge.

In [22]:
import pandas as pd

df_1 = pd.read_csv('First_run')
df_2 = pd.read_csv('Second_run')
df_3 = pd.read_csv('Third_run')

prior = pd.concat([df_1, df_2, df_3])


prior = prior.rename(columns = {'GradeLevel' : 'Prior_GradeLevel' , 'SchoolOfAttendance' : 'Prior_Calpads_Schools_2020_2021', 'EnrollmentStartDate' : 'Prior_EnrollmentStartDate', 'SchoolID': 'Prior_SchoolID'})

prior = prior.sort_values(by =['SSID', 'Prior_EnrollmentStartDate'])
prior = prior.drop_duplicates(subset = ['SSID'], keep = 'last' )
prior.reset_index(inplace = True, drop = True)

# prior school coming from calpads API call. 2020-2021 retains most recent enrollment

df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Downloaded_CSVS\Calpads_2021_2022.csv')

df['Start_Date'] = df['Start_Date'].astype(str)
df['Start_Date'] = df['Start_Date'].str[:10]
df['Start_Date'] = pd.to_datetime(df['Start_Date'])

most_recent = df[(df['Start_Date'] > '2021-07-30')]
most_recent = most_recent.rename(columns = {'SchoolCode': 'SchoolID', 'SchoolName': '8.1_School_2021_2022', 'SSID' : 'SSID', 'StudentName': 'StudentName', 'Grade_Level_Code': 'GradeLevel','Start_Date' : 'EnrollmentStartDate'})
most_recent.reset_index(inplace = True, drop = True)

# Filters Calpads 8.1 data to Start Dates after 2021-07-30 effectively getting 2021-2022 school year schools. 
# has one record per student

final = pd.merge(most_recent, prior, on = 'SSID')

final = final[['SSID', 'StudentName', 'SchoolID', '8.1_School_2021_2022', 'EnrollmentStartDate', 'GradeLevel', 'Prior_SchoolID', 'Prior_Calpads_Schools_2020_2021', 'Prior_EnrollmentStartDate', 'Prior_SchoolID', 'Prior_GradeLevel']]

# final.to_csv('Calpads_Prior_School_2020_2021', index = False)